In [3]:
import pandas as pd
import numpy as np
import os

In [59]:
# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
columns = ['SHRP_ID', 'State', 'Latitude', 'Longitude', 'Elevation', 'Road_class', 'Maint_date', 'Maint', 'IRI', 'IRI_date']

# Initialize an empty list to collect data
data = []

# Initialize lists to store data from different sheets
section_coords_list = []
shrp_info_list = []
iri_list = []

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Read the 'SECTION_COORDINATES' sheet to get SHRP_ID, Latitude, Longitude, and Elevation
        if 'SECTION_COORDINATES' in xls.sheet_names:
            section_coords_list.append(pd.read_excel(xls, sheet_name='SECTION_COORDINATES'))
        
        # Read the 'SHRP_INFO' sheet to get SHRP_ID and Road_class
        if 'SHRP_INFO' in xls.sheet_names:
            shrp_info_list.append(pd.read_excel(xls, sheet_name='SHRP_INFO'))
        
        # Read the 'IRI' sheet to get IRI data
        if 'IRI' in xls.sheet_names:
            iri_list.append(pd.read_excel(xls, sheet_name='IRI'))

# Concatenate all dataframes in the lists
if section_coords_list:
    section_coords_df = pd.concat(section_coords_list, ignore_index=True)
else:
    section_coords_df = pd.DataFrame()

if shrp_info_list:
    shrp_info_df = pd.concat(shrp_info_list, ignore_index=True)
else:
    shrp_info_df = pd.DataFrame()

if iri_list:
    iri_df = pd.concat(iri_list, ignore_index=True)
else:
    iri_df = pd.DataFrame()

# Merge SECTION_COORDINATES with SHRP_INFO on SHRP_ID
if not section_coords_df.empty and not shrp_info_df.empty:
    merged_df = pd.merge(section_coords_df, shrp_info_df, on='SHRP_ID', how='left')

    # Loop through each file in the directory again to collect the remaining data
    for filename in os.listdir(directory_path):
        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            file_path = os.path.join(directory_path, filename)
            
            # Load the Excel file
            xls = pd.ExcelFile(file_path)
            
            # Loop through each sheet in the Excel file
            for sheet_name in xls.sheet_names:
                if sheet_name not in ['SECTION_COORDINATES', 'SHRP_INFO', 'IRI']:  # Skip the processed sheets
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    
                    # Check if the sheet contains the 'SHRP_ID' column
                    if 'SHRP_ID' in df.columns:
                        
                        # Collect the data from the current sheet
                        for index, row in df.iterrows():
                            shrp_id = row['SHRP_ID']
                            
                            # Get corresponding merged data
                            merged_row = merged_df[merged_df['SHRP_ID'] == shrp_id]
                            if not merged_row.empty:
                                state = merged_row['STATE_CODE_EXP'].values[0] if 'STATE_CODE_EXP' in merged_row.columns else None
                                latitude = merged_row['LATITUDE'].values[0] if 'LATITUDE' in merged_row.columns else None
                                longitude = merged_row['LONGITUDE'].values[0] if 'LONGITUDE' in merged_row.columns else None
                                elevation = merged_row['ELEVATION'].values[0] if 'ELEVATION' in merged_row.columns else None
                                road_class = merged_row['FUNC_CLASS_EXP'].values[0] if 'FUNC_CLASS_EXP' in merged_row.columns else None
                            else:
                                state = None
                                latitude = None
                                longitude = None
                                elevation = None
                                road_class = None
                            
                            # Collect other data points from the row
                            maint_date = row['CN_ASSIGN_DATE'] if 'CN_ASSIGN_DATE' in row else None
                            maint = row['CN_CHANGE_REASON_EXP'] if 'CN_CHANGE_REASON_EXP' in row else None
                            iri = row['MRI'] if 'MRI' in row else None
                            iri_date = row['VISIT_DATE'] if 'VISIT_DATE' in row else None
                            
                            
                            # Append the collected data to the list
                            data.append({
                                'SHRP_ID': shrp_id,
                                'State': state,
                                'Latitude': latitude,
                                'Longitude': longitude,
                                'Elevation': elevation,
                                'Road_class': road_class,
                                'Maint_date': maint_date,
                                'Maint': maint,
                                'IRI': iri,
                                'IRI_date': iri_date
                            })

# Create a new DataFrame from the collected data
df6_key = pd.DataFrame(data, columns=columns)

       SHRP_ID State   Latitude  Longitude   Elevation  \
0         0163  None  37.608528  -99.23018  666.700012   
1         0260  None  33.442188 -112.69119  324.000000   
2         0261  None  33.453079 -112.73961  326.399994   
3         0669  None  41.211922  -86.26058  252.899994   
4         0114  None  36.658772  -79.36539  187.300003   
...        ...   ...        ...        ...         ...   
364726    0901  None  41.569759  -72.28599  130.399994   
364727    0902  None  41.565529  -72.24789   99.000000   
364728    0507  None  45.066380  -68.68948   46.700001   
364729    0502  None  45.056049  -68.68755   42.000000   
364730    1001  None  45.220230  -68.67458   66.400002   

                                   Road_class Maint_date Maint  IRI IRI_date  
0            Rural Principal Arterial - Other        NaT  None  NaN      NaT  
1       Rural Principal Arterial - Interstate        NaT  None  NaN      NaT  
2       Rural Principal Arterial - Interstate        NaT  None  Na

In [15]:
#df = df6_key.loc[~df6_key['Maint'].isnull()]
#df

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,PROJECT_LAYER_CODE,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,LAYER_COMMENT1,COMMENT_NOTE
2227,2227,1021,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,1994-05-01,Crack Sealing,NaN,...,NaN,10.0,Surface Treatment,AC,Asphalt concrete layer,0.00,Sand Seal,82.0,NaN,NaN
2228,2228,1021,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,1990-01-01,Crack Sealing,NaN,...,NaN,10.0,Surface Treatment,AC,Asphalt concrete layer,0.00,Sand Seal,82.0,NaN,NaN
2229,2229,6060,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,1993-01-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,2.54,Fog Seal,73.0,Z,MCC BEST JUSTEMENT
2230,2230,1017,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,2012-06-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,2.54,Chip Seal,71.0,Z,RUBBERIZED POROUS FRICTION COURSE
2231,2231,1018,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,2012-06-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,7.62,Chip Seal,71.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74044,74044,1024,Arizona,35.296162,-83.18010,647.599976,Rural Principal Arterial - Interstate,2018-07-01,"Patch Pot Holes - Hand Spread, Compacted with ...",NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,0.00,Fog Seal,73.0,NaN,NaN
74045,74045,1096,Texas,29.355900,-98.83502,235.000000,Rural Principal Arterial - Other,2017-01-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,0.00,Fog Seal,73.0,NaN,NaN
74046,74046,1096,Texas,29.355900,-98.83502,235.000000,Rural Principal Arterial - Other,2013-01-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,0.00,Fog Seal,73.0,NaN,NaN
74047,74047,1096,Texas,29.355900,-98.83502,235.000000,Rural Principal Arterial - Other,1995-01-01,Crack Sealing,NaN,...,NaN,2.0,Seal Coat,AC,Asphalt concrete layer,0.00,Fog Seal,73.0,NaN,NaN


In [16]:
df6_key.columns

Index(['Unnamed: 0', 'SHRP_ID', 'State', 'Latitude', 'Longitude', 'Elevation',
       'Road_class', 'Maint_date', 'Maint', 'IRI', 'IRI_date',
       'HPMS16_CRACKING_PERCENT_AC', 'MEPDG_CRACKING_PERCENT_AC',
       'MEPDG_TRANS_CRACK_LENGTH_AC', 'MEPDG_LONG_CRACK_LENGTH_AC',
       'ME_PERCENT_WHEEL_PATH_CRACK', 'SURVEY_WIDTH', 'GATOR_CRACK_A',
       'GATOR_CRACK_A_L', 'GATOR_CRACK_A_M', 'GATOR_CRACK_A_H', 'BLK_CRACK_A',
       'BLK_CRACK_A_L', 'SURVEY_DATE', 'EDGE_CRACK_L', 'LONG_CRACK_WP_L',
       'LONG_CRACK_NWP_L', 'LONG_CRACK_WP_SEAL_L', 'LONG_CRACK_NWP_SEAL_L',
       'TRANS_CRACK_NO', 'TRANS_CRACK_L', 'TRANS_CRACK_SEAL_L', 'PATCH_A',
       'POTHOLES_A', 'SHOVING_A', 'BLEEDING', 'RAVELING', 'CONSTRUCTION_NO',
       'LAYER_NO', 'PROJECT_LAYER_CODE', 'DESCRIPTION', 'DESCRIPTION_EXP',
       'LAYER_TYPE', 'LAYER_TYPE_EXP', 'REPR_THICKNESS', 'MATL_CODE_EXP',
       'MATL_CODE', 'LAYER_COMMENT1', 'COMMENT_NOTE'],
      dtype='object')

In [18]:
df6_key[df6_key['MEPDG_CRACKING_PERCENT_AC']>1]

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,PROJECT_LAYER_CODE,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,LAYER_COMMENT1,COMMENT_NOTE
1,1,0260,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,D,5.0,Base Layer,GB,Unbound (granular) base,101.60,Crushed Gravel,304.0,NaN,NaN
2,2,0261,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,D,5.0,Base Layer,GB,Unbound (granular) base,101.60,Crushed Gravel,304.0,NaN,NaN
5,5,0119,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,H,2.0,Seal Coat,AC,Asphalt concrete layer,5.08,Chip Seal,71.0,Z,INFO FROM 2014 REHAB EVENT
6,6,0124,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,H,2.0,Seal Coat,AC,Asphalt concrete layer,5.08,Chip Seal,71.0,Z,INFO FROM 2014 REHAB EVENT
8,8,0113,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,N,2.0,Seal Coat,AC,Asphalt concrete layer,12.70,Slurry Seal,72.0,Q,THICKNESS FROM ADJACENT SECTIONS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364711,364711,A310,Utah,NaN,NaN,NaN,Rural Minor Arterial,NaN,NaN,NaN,...,K,2.0,Seal Coat,AC,Asphalt concrete layer,10.16,Slurry Seal,72.0,T,NaN
364714,364714,A320,Utah,NaN,NaN,NaN,Rural Minor Arterial,NaN,NaN,NaN,...,J,2.0,Seal Coat,AC,Asphalt concrete layer,7.62,Slurry Seal,72.0,T,NaN
364716,364716,0902,Mississippi,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,E,8.0,Interlayer,AC,Asphalt concrete layer,2.54,Stress Absorbing Membrane Interlayer,77.0,NaN,NaN
364720,364720,A320,Utah,NaN,NaN,NaN,Rural Minor Arterial,NaN,NaN,NaN,...,J,2.0,Seal Coat,AC,Asphalt concrete layer,7.62,Slurry Seal,72.0,T,NaN


In [75]:
#columns_to_drop = ['SURVEY_DATE','HPMS16_CRACKING_PERCENT_AC', 'MEPDG_CRACKING_PERCENT_AC', 'MEPDG_TRANS_CRACK_LENGTH_AC',
                      'MEPDG_LONG_CRACK_LENGTH_AC', 'ME_PERCENT_WHEEL_PATH_CRACK', 'SURVEY_WIDTH', 'GATOR_CRACK_A',
                      'GATOR_CRACK_A_L', 'GATOR_CRACK_A_M', 'GATOR_CRACK_A_H', 'BLK_CRACK_A', 'BLK_CRACK_A_L']
#df6_key = df_key.drop(columns_to_drop, axis=1)

In [84]:
 """
    Data on defects. Fetch data on the different defects collected during road condition surveys.
    :param SURVEY_DATE: the date the road condition was assessed. This will be essential in comparing with IRI reported.
    :param HPMS16_CRACKING_PERCENT_AC: Percent of section cracked using 2016 HPMS Field Guide definitions. Includes only 
        longitudinal and fatigue cracking in assumed 1m wide wheel paths. Units: %
    :param MEPDG_CRACKING_PERCENT_AC: The total area of alligator cracking summed across all levels of severity, divided 
        by the total area of the test section, in accordance with MEPDG definitions. Note that LTPP alligator cracking 
        interpretations are not restricted to the wheel path.
        Units: %
    :param MEPDG_TRANS_CRACK_LENGTH_AC: Total length of sealed and unsealed transverse cracks at all levels of severity, 
        divided by test section length in accordance with current pavement MEPDG distress definitions.
        Units: m/km
    :param MEPDG_LONG_CRACK_LENGTH_AC: Total length of sealed and unsealed longitudinal cracks in the wheel path at all 
        severity levels divided by test section length.
        Units: m/km
    :param ME_PERCENT_WHEEL_PATH_CRACK: Percentage of cracking in assumed 0.61 meter wide wheel paths corresponding to 
        the MEPDG_CRACKING_PERCENT_AC field reported in the 2016 and 2017 LTPP data releases.
        Units: %
    :param SURVEY_WIDTH: Width of pavement surveyed for distress measurements. Units: m
    :param GATOR_CRACK_A: Total alligator (fatigue) cracking area. Units: sq.m
    :param BLK_CRACK_A: Total area of block cracking. Units: sq.m
    :param EDGE_CRACK_L: Total length of edge cracking. Units: m
    :param LONG_CRACK_WP_L: Total longitudinal cracking length in wheel path. Units: m
    :param LONG_CRACK_NWP_L: Total length of non-wheel path longitudinal cracking. Units: m
    :param LONG_CRACK_WP_SEAL_L: Total well sealed longitudinal cracking length in wheel path. Units: m
    :param LONG_CRACK_NWP_SEAL_L: Total well sealed non-wheel longitudinal cracking length in wheel path. Units: m 
    :param TRANS_CRACK_L: Total transverse cracking length. Units: m
    :param TRANS_CRACK_SEAL_L:Total length of well sealed transverse cracking. Units: m
    :param PATCH_A: Total area of patching. Units: sq.m
    :param POTHOLES_A: Total pothole area. Units: sq.m
    :param SHOVING_A: Area of shoving, localized longitudinal displacement of the pavement surface. Units: sq.m
    :param BLEEDING: Presence of excess asphalt on the pavement surface which may create a shiny, glass-like reflective surface. 
        Units: sq.m
    :param RAVELING: Wearing away of the pavement surface caused by the dislodging of aggregate particles and loss of asphalt binder.
        Units: sq.m 
    :param MAX_MEAN_DEPTH_1_8: Maximum Average Depth of rutting From 1.8m Straight Edge. Units: mm
    :param MAX_MEAN_DEPTH_WIRE_REF: Maximum Average Depth of rutting From Wire Reference. Units: mm
    """
# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['SURVEY_DATE','HPMS16_CRACKING_PERCENT_AC', 'MEPDG_CRACKING_PERCENT_AC', 'MEPDG_TRANS_CRACK_LENGTH_AC',
                      'MEPDG_LONG_CRACK_LENGTH_AC', 'ME_PERCENT_WHEEL_PATH_CRACK', 'SURVEY_WIDTH', 'GATOR_CRACK_A',
                      'BLK_CRACK_A', 'EDGE_CRACK_L', 'LONG_CRACK_WP_L', 'LONG_CRACK_NWP_L', 'LONG_CRACK_WP_SEAL_L',
                      'LONG_CRACK_NWP_SEAL_L', 'TRANS_CRACK_NO', 'TRANS_CRACK_L', 'TRANS_CRACK_SEAL_L', 'PATCH_A',
                      'POTHOLES_A', 'SHOVING_A', 'BLEEDING', 'RAVELING', 'MAX_MEAN_DEPTH_WIRE_REF', 'MAX_MEAN_DEPTH_1_8']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'SHRP_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['SHRP_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df_key['SHRP_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['SHRP_ID'] == shrp_id, col] = value

In [ ]:
 """
    Data on Pavement layers. Fetch data on the the type of and thickness of the pavement layers.
    :param CLIMATIC_RS: 
    :param CONSTRUCTION_NO: Event number used to relate changes in pavement structure with other time dependent data elements. 
        This field is set to 1 when a test section is initially accepted into LTPP and is incremented with each change to the 
        layer structure.
    :param LAYER_NO: Unique sequential number assigned to pavement layers, starting with layer 1 as the deepest layer (subgrade).
    :param PROJECT_LAYER_CODE: Sequential alphabetic code assigned to identify group project wide layers.
    :param DESCRIPTION: Code indicating general type of layer.
    :param DESCRIPTION_EXP: Code indicating general type of layer. (Code Expansion)
    :param LAYER_TYPE: Code identifying the type of layer.
    :param LAYER_TYPE_EXP: Code identifying the type of layer. (Code Expansion)
    :param REPR_THICKNESS: Representative thickness for a layer in a section. Units: mm
    :param MATL_CODE_EXP: TCode corresponding to the material type classification. (Code Expansion)
    :param MATL_CODE: Code corresponding to the material type classification.
    :param LAYER_COMMENT1: Codes to describe any additional information concerning layer.
    :param COMMENT_NOTE: Additional comments about layer.
    :param RESILIENT_MODULUS: Resilient modulus calculated from laboratory testing per AASHTO T 307 at assumed stress 
        values in the field. Units: MPa
    """
# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['CONSTRUCTION_NO','LAYER_NO', 'PROJECT_LAYER_CODE', 'DESCRIPTION', 'DESCRIPTION_EXP',	'LAYER_TYPE',
                      'LAYER_TYPE_EXP',	'REPR_THICKNESS', 'MATL_CODE_EXP', 'MATL_CODE',	'LAYER_COMMENT1', 'COMMENT_NOTE', 
                      'RESILIENT_MODULUS']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'SHRP_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['SHRP_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df_key['SHRP_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['SHRP_ID'] == shrp_id, col] = value

In [20]:
"""
Collect MERRA_ID data inorder to now read the MERRA data
"""

# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['MERRA_ID']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'SHRP_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['SHRP_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df6_key['SHRP_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['SHRP_ID'] == shrp_id, col] = value

In [ ]:
"""
Collect weather data from MERRA. The MERRA_ID is linked to a SHRP_ID. 
    :param SHRP_ID
    :param MERRA_ID
    :param YEAR
    :param SURVEY_DATE
    :param PRECIPITATION
    :param EVAPORATION	
    :param PRECIP_DAYS
    :param MONTH
    :param TEMP_MEAN_AVG	
    :param FREEZE_INDEX
    :param FREEZE_THAW
    :param HIGH_TEMP_7DAYS
    :param WIND_VELOCITY_AVG
    :param REL_HUM_AVG_AVG  
    :param YEAR
"""


# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['YEAR', 'PRECIPITATION', 'EVAPORATION', 'PRECIP_DAYS', 'MONTH', 'TEMP_MEAN_AVG', 'FREEZE_INDEX', 'FREEZE_THAW', 'HIGH_TEMP_7DAYS']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'MERRA_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['MERRA_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df6_key['MERRA_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['MERRA_ID'] == shrp_id, col] = value

In [26]:
df6_key.columns

Index(['Unnamed: 0', 'SHRP_ID', 'State', 'Latitude', 'Longitude', 'Elevation',
       'Road_class', 'Maint_date', 'Maint', 'IRI', 'IRI_date',
       'HPMS16_CRACKING_PERCENT_AC', 'MEPDG_CRACKING_PERCENT_AC',
       'MEPDG_TRANS_CRACK_LENGTH_AC', 'MEPDG_LONG_CRACK_LENGTH_AC',
       'ME_PERCENT_WHEEL_PATH_CRACK', 'SURVEY_WIDTH', 'GATOR_CRACK_A',
       'GATOR_CRACK_A_L', 'GATOR_CRACK_A_M', 'GATOR_CRACK_A_H', 'BLK_CRACK_A',
       'BLK_CRACK_A_L', 'SURVEY_DATE', 'EDGE_CRACK_L', 'LONG_CRACK_WP_L',
       'LONG_CRACK_NWP_L', 'LONG_CRACK_WP_SEAL_L', 'LONG_CRACK_NWP_SEAL_L',
       'TRANS_CRACK_NO', 'TRANS_CRACK_L', 'TRANS_CRACK_SEAL_L', 'PATCH_A',
       'POTHOLES_A', 'SHOVING_A', 'BLEEDING', 'RAVELING', 'CONSTRUCTION_NO',
       'LAYER_NO', 'PROJECT_LAYER_CODE', 'DESCRIPTION', 'DESCRIPTION_EXP',
       'LAYER_TYPE', 'LAYER_TYPE_EXP', 'REPR_THICKNESS', 'MATL_CODE_EXP',
       'MATL_CODE', 'LAYER_COMMENT1', 'COMMENT_NOTE', 'MERRA_ID',
       'PRECIPITATION', 'EVAPORATION', 'PRECIP_DAYS', 'MO

In [44]:
df6_key[df6_key['SHRP_ID']=='0608']

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,DRAINAGE_TYPE,DRAINAGE_TYPE_EXPOTHER_DRAINAGE_TYPE,DRAINPIPE_DIAMETER,LATERALS_SPACING,RESILIENT_MODULUS,MAX_MEAN_DEPTH_WIRE_REF,MAX_MEAN_DEPTH_1_8,TEST_DATE,UNLOAD_PEAK_DEFLECTION,PEAK_DEFL_1
39,39,0608,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
349,349,0608,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
350,350,0608,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
351,351,0608,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
352,352,0608,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363697,363697,0608,California,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
363730,363730,0608,California,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
363943,363943,0608,California,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None
364308,364308,0608,California,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,4,4,None,None,None


In [28]:
 """
 COLLECT DRAINAGE DATA.
    :param DRAINAGE_LOCATION: Code indicating if subsurface drainage is continuous along section or at intermittent 
        locations or not provided.
    :param DRAINAGE_LOCATION_EXP: Code indicating if subsurface drainage is continuous along section or at intermittent 
        locations or not provided. (Code Expansion)
    :param DRAINAGE_TYPE: Code indicating type of subsurface drainage.
    :param DRAINAGE_TYPE_EXP: Code indicating type of subsurface drainage. (Code Expansion)
    :param OTHER_DRAINAGE_TYPE: Drainage type if other than those specified.
    :param DRAINPIPE_DIAMETER: Inside diameter of longitudinal drainpipes used for subsurface drainage. Unit: mm
    :param LATERALS_SPACING: Average spacing for subdrainage laterals. Unit: m
    """
# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['DRAINAGE_LOCATION', 'DRAINAGE_LOCATION_EXP', 'DRAINAGE_TYPE', 'DRAINAGE_TYPE_EXP' 'OTHER_DRAINAGE_TYPE', 
                      'DRAINPIPE_DIAMETER', 'LATERALS_SPACING', 'RESILIENT_MODULUS', 'MAX_MEAN_DEPTH_WIRE_REF', 'MAX_MEAN_DEPTH_1_8']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'SHRP_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['SHRP_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df6_key['SHRP_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['SHRP_ID'] == shrp_id, col] = value

In [47]:
 """
    Field test data on pavement structural integrity. Fetch data on the test results from Falling Weight Deflectometer.
    :param TEST_DATE: Date the test was performed
    :param UNLOAD_PEAK_DEFLECTION: Peak deflection value on the unloaded side of the joint/crack used for the LTE 
        computation. Units: microns
    :param PEAK_DEFL_1: Peak deflection value from the sensor located at the center of the load plate on the loaded 
        side of the joint/crack used for the LTE computation. Usually considered as the maximum deflection. Units: microns
    """
# Define the path to the directory containing the Excel files
directory_path = '/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Bucket'

# Define the columns for the new DataFrame
additional_columns = ['TEST_DATE','UNLOAD_PEAK_DEFLECTION','PEAK_DEFL_1']

# Ensure the additional columns are added to the existing DataFrame with None values
for col in additional_columns:
    df6_key[col] = None

# Initialize lists to store data for new columns
additional_data = {col: [] for col in additional_columns}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the Excel file
        xls = pd.ExcelFile(file_path)
        
        # Loop through each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Check if the sheet contains the 'SHRP_ID' column
            if 'SHRP_ID' in df.columns:
                
                # Collect the data for the new columns from the current sheet
                for index, row in df.iterrows():
                    shrp_id = row['SHRP_ID']
                    
                    # Check if the SHRP_ID is in the existing DataFrame
                    if shrp_id in df6_key['SHRP_ID'].values:
                        for col in additional_columns:
                            if col in row:
                                additional_data[col].append((shrp_id, row[col]))

# Add the additional data to the DataFrame
for col in additional_columns:
    for shrp_id, value in additional_data[col]:
        df6_key.loc[df6_key['SHRP_ID'] == shrp_id, col] = value

In [48]:
df6_key[df6_key['PEAK_DEFL_1']>0]

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,DRAINAGE_TYPE,DRAINAGE_TYPE_EXPOTHER_DRAINAGE_TYPE,DRAINPIPE_DIAMETER,LATERALS_SPACING,RESILIENT_MODULUS,MAX_MEAN_DEPTH_WIRE_REF,MAX_MEAN_DEPTH_1_8,TEST_DATE,UNLOAD_PEAK_DEFLECTION,PEAK_DEFL_1
7,7,0606,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,54.471,3,3,1998-02-09 00:00:00,111.0,168
11,11,0604,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,54.471,2,2,1991-09-24 00:00:00,175.0,200
12,12,0604,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,54.471,2,2,1991-09-24 00:00:00,175.0,200
13,13,0661,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,66.192,2,2,2001-05-23 00:00:00,251.0,263
14,14,0665,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,66.192,4,4,1993-05-13 00:00:00,258.0,291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364689,364689,3813,Washington,NaN,NaN,NaN,Urban Other Principal Arterial,NaN,NaN,NaN,...,1.0,None,NaN,NaN,62.745,8,8,2002-04-23 00:00:00,281.0,305
364690,364690,0661,California,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,66.192,2,2,2001-05-23 00:00:00,251.0,263
364716,364716,0902,Mississippi,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,1,1,1992-05-27 00:00:00,184.0,297
364726,364726,0901,New Mexico,NaN,NaN,NaN,Rural Local Collector,NaN,NaN,NaN,...,1.0,None,NaN,NaN,63.434,7,7,1992-04-28 00:00:00,220.0,231


In [50]:
# Define function to calculate Deflection Basin Ratio (DBR)
def calculate_DBR(row):
    """
    Calculate Deflection Basin Ratio.
    :param PEAK_DEFL_1: Peak deflection value from the sensor located at the center of the load plate on the loaded 
        side of the joint/crack used for the LTE computation. Usually considered as the maximum deflection.
        Units: microns
    :param UNLOAD_PEAK_DEFLECTION: Peak deflection value on the unloaded side of the joint/crack used for the LTE 
        computation. Units: microns
    :param DBR: Deflection Basin Ratio. Ratio of the peak deflection at the unloaded side (typically a
        distance of 900-1200mm from the load center) to the maximum deflection (peak deflection at centre).  Higher 
        DBR (closer to 1) indicates a more flexible pavement structure.
    :return: DBR value. Higher DBR (closer to 1) indicates a more flexible pavement structure.
    """
    PEAK_DEFL_1 = row['PEAK_DEFL_1']
    UNLOAD_PEAK_DEFLECTION = row['UNLOAD_PEAK_DEFLECTION']
    
    if UNLOAD_PEAK_DEFLECTION is None or PEAK_DEFL_1 is None or PEAK_DEFL_1 == 0:
        return None
    else:
        return UNLOAD_PEAK_DEFLECTION / PEAK_DEFL_1

# Define function to calculate Curvature Index (CI)
def calculate_CI(row):
    """
    Calculate Curvature Index.
    :param PEAK_DEFL_1: Peak deflection value from the sensor located at the center of the load plate on the loaded 
        side of the joint/crack used for the LTE computation. Usually considered as the maximum deflection.
        Units: microns
    :param UNLOAD_PEAK_DEFLECTION: Peak deflection value on the unloaded side of the joint/crack used for the LTE 
        computation. Units: microns
    :param CI: Curvature Index. Calculated as the difference between the maximum delection and deflection at a specific
        distance from the load center (typically 300-600mm). A higher CI indicates a more pronounced deflection basin
        implying a weaker pavement structure.
    :return: CI value. A higher CI indicates a more pronounced deflection basin implying a weaker pavement structure.
    """
    PEAK_DEFL_1 = row['PEAK_DEFL_1']
    UNLOAD_PEAK_DEFLECTION = row['UNLOAD_PEAK_DEFLECTION']
    
    if UNLOAD_PEAK_DEFLECTION is None or PEAK_DEFL_1 is None:
        return None
    else:
        return PEAK_DEFL_1 - UNLOAD_PEAK_DEFLECTION

# Assuming df6_key is your DataFrame
# Apply the functions to each row in the DataFrame
df6_key['DBR'] = df6_key.apply(calculate_DBR, axis=1)
df6_key['CI'] = df6_key.apply(calculate_CI, axis=1)

In [51]:
df6_key[df6_key['DRAINPIPE_DIAMETER']>0]

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,DRAINPIPE_DIAMETER,LATERALS_SPACING,RESILIENT_MODULUS,MAX_MEAN_DEPTH_WIRE_REF,MAX_MEAN_DEPTH_1_8,TEST_DATE,UNLOAD_PEAK_DEFLECTION,PEAK_DEFL_1,DBR,CI
5,5,0119,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,152.4,76.2,93.083,2,2,None,None,None,NaN,NaN
6,6,0124,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,152.4,76.2,44.818,5,5,None,None,None,NaN,NaN
30,30,0119,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,152.4,76.2,93.083,2,2,None,None,None,NaN,NaN
31,31,0119,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,152.4,76.2,93.083,2,2,None,None,None,NaN,NaN
32,32,0120,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,152.4,76.2,140.658,15,13,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364635,364635,9267,Illinois,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,101.6,152.4,46.886,2,1,1993-08-16 00:00:00,162.0,175,0.925714,13.0
364642,364642,3009,Iowa,NaN,NaN,NaN,Urban Principal Arterial - Interstate,NaN,NaN,NaN,...,152.4,121.92,135.832,4,3,2014-05-21 00:00:00,160.0,174,0.919540,14.0
364643,364643,5843,Illinois,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,101.6,152.4,101.357,3,1,2002-04-30 00:00:00,204.0,294,0.693878,90.0
364661,364661,A909,Wisconsin,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,101.6,NaN,84.809,7,3,1992-06-11 00:00:00,144.0,161,0.894410,17.0


In [46]:
to_drop = ['UNLOAD_PEAK_DEFLECTION', 'PEAK_DEFL_1']
df6_key = df6_key.drop(to_drop, axis=1)
#df6_key = df6_key.drop(to_drop, inplace=True)

In [ ]:
def calculate_unit_defect ()
    """
    Calculate Unit change in defects. Change in defect from previous recording. 
    :param IRI_diff: Change in IRI from one recording time to another
    :param Unit_IRI: Unit change in IRI per annum. Normalized 
    :param TTCrack_diff: Change in Percent of section cracked using 2016 HPMS Field Guide definitions. Includes only 
        longitudinal and fatigue cracking in assumed 1m wide wheel paths. Units: %
    :param Unit_TTCrack : Unit change in total cracking per annum
    :param Allig_diff: Change in the total area of alligator cracking from one recording period to another. Units: %
    :param Unit_Allig: Unit change in total alligator cracks per annum
    :param Trans_diff: Change in the total length of sealed and unsealed transverse cracks at all levels of severity, 
        divided by test section length in accordance with current pavement MEPDG distress definitions. Units: m/km
    :param Unit_Trans: Unit change in transverse cracks per annum
    :param MEPDG_LONG_CRACK_LENGTH_AC: Total length of sealed and unsealed longitudinal cracks in the wheel path at all 
        severity levels divided by test section length.
        Units: m/km
    :param ME_PERCENT_WHEEL_PATH_CRACK: Percentage of cracking in assumed 0.61 meter wide wheel paths corresponding to 
        the MEPDG_CRACKING_PERCENT_AC field reported in the 2016 and 2017 LTPP data releases.
        Units: %
    :param SURVEY_WIDTH: Width of pavement surveyed for distress measurements. Units: m
    :param GATOR_CRACK_A: Total alligator (fatigue) cracking area. Units: sq.m
    :param BLK_CRACK_A: Total area of block cracking. Units: sq.m
    :param EDGE_CRACK_L: Total length of edge cracking. Units: m
    :param LONG_CRACK_WP_L: Total longitudinal cracking length in wheel path. Units: m
    :param LONG_CRACK_NWP_L: Total length of non-wheel path longitudinal cracking. Units: m
    :param LONG_CRACK_WP_SEAL_L: Total well sealed longitudinal cracking length in wheel path. Units: m
    :param LONG_CRACK_NWP_SEAL_L: Total well sealed non-wheel longitudinal cracking length in wheel path. Units: m 
    :param TRANS_CRACK_L: Total transverse cracking length. Units: m
    :param TRANS_CRACK_SEAL_L:Total length of well sealed transverse cracking. Units: m
    :param PATCH_A: Total area of patching. Units: sq.m
    :param POTHOLES_A: Total pothole area. Units: sq.m
    :param SHOVING_A: Area of shoving, localized longitudinal displacement of the pavement surface. Units: sq.m
    :param BLEEDING: Presence of excess asphalt on the pavement surface which may create a shiny, glass-like reflective surface. 
        Units: sq.m
    :param RAVELING: Wearing away of the pavement surface caused by the dislodging of aggregate particles and loss of asphalt binder.
        Units: sq.m 
    """
    

In [55]:
df6_key

,Unnamed: 0,SHRP_ID,State,Latitude,Longitude,Elevation,Road_class,Maint_date,Maint,IRI,...,DRAINPIPE_DIAMETER,LATERALS_SPACING,RESILIENT_MODULUS,MAX_MEAN_DEPTH_WIRE_REF,MAX_MEAN_DEPTH_1_8,TEST_DATE,UNLOAD_PEAK_DEFLECTION,PEAK_DEFL_1,DBR,CI
0,0,0163,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,NaN,NaN,132.384,15,15,None,None,None,NaN,NaN
1,1,0260,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,NaN,NaN,52.402,9,8,None,None,None,NaN,NaN
2,2,0261,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,NaN,NaN,75.845,8,7,None,None,None,NaN,NaN
3,3,0669,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,NaN,NaN,66.882,5,5,None,None,None,NaN,NaN
4,4,0114,Arizona,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,NaN,NaN,139.969,6,5,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364726,364726,0901,New Mexico,NaN,NaN,NaN,Rural Local Collector,NaN,NaN,NaN,...,NaN,NaN,63.434,7,7,1992-04-28 00:00:00,220.0,231,0.952381,11.0
364727,364727,0902,Mississippi,NaN,NaN,NaN,Rural Principal Arterial - Interstate,NaN,NaN,NaN,...,NaN,NaN,63.434,1,1,1992-05-27 00:00:00,184.0,297,0.619529,113.0
364728,364728,0507,Florida,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,NaN,NaN,115.147,2,2,None,None,None,NaN,NaN
364729,364729,0502,Florida,NaN,NaN,NaN,Rural Principal Arterial - Other,NaN,NaN,NaN,...,NaN,NaN,127.558,6,4,None,None,None,NaN,NaN


In [57]:
df6_key.to_csv('/Users/pamelaacheng/Library/CloudStorage/OneDrive-Nexus365/DPhil/2022/HDM4/Data/Road condition/LTTP/Road_data_310524.csv')

In [56]:
# Check for NaN values
print(df6_key.isna().sum())


Unnamed: 0                     0
SHRP_ID                        0
State                       9328
Latitude                   40145
Longitude                  40145
                           ...  
TEST_DATE                 291656
UNLOAD_PEAK_DEFLECTION    291656
PEAK_DEFL_1               291656
DBR                       291656
CI                        291656
Length: 72, dtype: int64
